In [2]:
pip install pingouin

     -------------------------------------- 198.6/198.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=f21560ccfd9f04dcffb07dee87ce432da63cbe5a6f9878f8b00d559e8b75dea1
  Stored in directory: c:\users\ibrahim\appdata\local\pip\cache\wheels\04\bb\0d\2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built littleutils
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
import scipy.stats as stats
import scipy.stats as st
import pingouin as pg

# Import des données clients

In [ ]:
# Import du et première vu du fichier client 

datacustomers = pd.read_csv("Data/raw_data/customers.csv")
datacustomers

# Vérification des données clients

In [ ]:
# Info du fichier client

datacustomers.info()

In [ ]:
# Recherche des valeurs null

print(datacustomers.isnull().sum())

In [ ]:
# Vérification des genres

datacustomers['sex'].unique()

In [ ]:
# Recherche des doublons

datacustomers.loc[datacustomers['client_id'].duplicated(keep=False),:]

In [ ]:
# Recherche de clients test

client_test = datacustomers[datacustomers['client_id'].str.contains('ct')]
client_test

# Nettoyage des données clients

In [ ]:
datacustomers = datacustomers.drop(client_test.index)

# Import des données produits

In [ ]:
# Import et première vu du fichier produit

dataproducts = pd.read_csv("Data/raw_data/products.csv")
dataproducts

# Vérification des données produits

In [ ]:
# Infos du fichier produit

dataproducts.info()

In [ ]:
# Recherche des valeurs null

print(dataproducts.isnull().sum())

In [ ]:
# Verification des catégories éxistantes

dataproducts['categ'].unique()

In [ ]:
# Vérification du prix maximum

dataproducts['price'].max()

In [ ]:
# Vérification du prix minimum

dataproducts['price'].min()

In [ ]:
# Recherche de l'erreur de prix minimum

dataproducts[dataproducts['price'] == -1.0]

In [ ]:
#Recherche des 'id_prod' commencant par la lettre 'T' (les testes)

prod_test = dataproducts[dataproducts['id_prod'].str.contains('T','t')]
prod_test

In [ ]:
# Recherche des doublons

dataproducts.loc[dataproducts['id_prod'].duplicated(keep=False),:]

# Nettoyage des données produits

In [ ]:
# Suppression des produits test

dataproducts = dataproducts.drop(prod_test.index)

# Import des données de transaction

In [ ]:
# Import et première vu du fichier transaction

datatransactions = pd.read_csv("Data/raw_data/transactions.csv")
datatransactions

# Vérification des données de transaction

In [ ]:
# Info du DataFrame

datatransactions.info()

In [ ]:
# Recherche des valeurs null

print(datatransactions.isnull().sum())

In [ ]:
# Recherche des doublons

datatransactions.loc[datatransactions[['session_id','id_prod','client_id', 'date']].duplicated(keep=False),:]

In [ ]:
# Classement des dates dans l'ordre décroissant

datatransactions.sort_values(by=['date'], ascending=False)

In [ ]:
#Recherche des 'date' commencant par 'T'

datatransactions[datatransactions['date'].str.contains('t', 'T')]

# Nettoyage des données transactions

In [ ]:
datatransactions = datatransactions.drop(datatransactions[datatransactions['id_prod'] == 'T_0'].index)

In [ ]:
datatransactions['date'] = pd.to_datetime(datatransactions['date']).dt.normalize()
datatransactions['date']

In [ ]:
datatransactions


# Merge
---

In [ ]:
# Jointure test pour constater les données manquantes

dataV2 = pd.merge(dataproducts, datatransactions, how="outer", on=["id_prod"])
dataV2

In [ ]:
Val_nul = dataV2[dataV2['price'].isnull()]
Val_nul.nunique()

**On remarque un produit vendu 221 fois mais sans prix ni catégorie !**

- Cependant on constate que l'id_prod est construit a l'aide de la catégorie (le premier chiffre représente la catégorie)


In [ ]:
# Recherche du prix moyen des produits de la catégorie 0

mean_price = dataproducts.groupby('categ').mean().reset_index()
mean_price = mean_price[mean_price['categ'] == 0]
mean_price = round(mean_price['price'])
mean_price

In [ ]:
# Ajout du produit manquant dans le fichier produit avec les données recolté (catégorie + prix moyen)

dataproducts = dataproducts.append({'id_prod':'0_2245','price':12.0,'categ': 0}, ignore_index=True)

In [ ]:
dataproducts[dataproducts['id_prod'] == '0_2245']

In [ ]:
# Vérification des valeurs nulles

dataV2.isnull().sum()

- On constate qu'il y'a 21 produits non vendu !

In [ ]:
# Jointure test pour constater les données manquantes

dataV3 = pd.merge(datacustomers, datatransactions, how="outer", on=["client_id"])
dataV3

In [ ]:
# Recherche des valeurs nulles

dataV3.isnull().sum()

- On constate qu'il y'a 21 clients inactif !

In [ ]:
dataV1 = pd.merge(dataproducts, datatransactions, how="inner", on=["id_prod"])
dataV1

In [ ]:
# Reprise du DataFrame avec une jointure interne pour ne pas prendre les clients inactif et les produits non-vendu !

data = pd.merge(dataV1, datacustomers, how="inner", on=["client_id"])
data

## *Calcul et ajout de l'age*

In [ ]:
data = data.assign(age = 2023 - data['birth'])
data

## *Classement par groupe d'age*

In [ ]:
# Création d'une colonne 'Groupe' et ajout du nom du groupe

data['Groupe'] = 'test'

data.loc[data['age'] < 30,['Groupe']] = '- 30 ans'
data.loc[(data['age'] >= 30) & (data['age'] < 50),['Groupe']] = '30-50 ans'
data.loc[(data['age'] >= 50) & (data['age'] < 70),['Groupe']] = '50-70 ans'
data.loc[(data['age'] >= 70),['Groupe']] = '+ 70 ans'

# Mettre les différents groupe dans l'ordre croissants

order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
data = data.reindex(data['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)
data

In [ ]:
# Vérification des groupes d'age

data[data['Groupe'] == 'test']

# <center>**Demandes d'Antoine**
---

## <center>Analyse des tendance du CA

### *Chiffre d'affaire total*

In [ ]:
CA = round(data['price'].sum(),2)
print("Chiffre d'affaire total :", CA, "€")

### *Chiffre d'affaire dans le temp*

In [ ]:
CAtemp = data.groupby(by=['date']).sum().reset_index()
CAtemp['date'] = pd.to_datetime (CAtemp['date'])
CAtemp = CAtemp.rename(columns={'price':'CA'})
CAtemp['CA'] = round(CAtemp['CA'])
CAtemp = CAtemp[['date','CA']]
CAtemp

In [ ]:
CAtemp.plot.line(y='CA', x = 'date',figsize=(15, 8))
plt.title("Etude du chiffre d'affaire dans le temps", size = 20)
plt.ylabel("Chiffre d'affaire", fontsize= 15)
plt.xlabel("Temps", fontsize= 15)
plt.show()

In [ ]:
# Décomposition en moyenne mobile

CAmoymobile = CAtemp
CAmoymobile['CA'] = CAmoymobile['CA'].rolling(4).mean()
CAmoymobile.plot.line(y='CA', x = 'date',figsize=(15, 8))
plt.title("Etude du chiffre d'affaire dans le temps", size = 20)
plt.ylabel("Chiffre d'affaire", fontsize= 15)
plt.xlabel("Temps", fontsize= 15)
plt.show()

- Légère augmentation mois de septembre de l'année 2021 !
- Grosse baisse du mois d'octobre au mois de novembre de l'année 2021 !
- Légère augmentation mois de février de l'année 2022 !

In [ ]:
Oct_2021 = data
Oct_2021['year'] = pd.DatetimeIndex(Oct_2021['date']).year.astype(str)
Oct_2021['month'] = pd.DatetimeIndex(Oct_2021['date']).month.astype(str)
Oct_2021['Mois-Année'] = Oct_2021['month']+'-'+Oct_2021['year']
Oct_2021 = Oct_2021[Oct_2021['Mois-Année'] == '10-2021']
CA_Categ_Oct_2021 = Oct_2021.groupby('categ').sum()
CA_Categ_Oct_2021 = CA_Categ_Oct_2021[['price']].reset_index()
CA_Categ_Oct_2021 = CA_Categ_Oct_2021.rename(columns={'price':'CA'})
CA_Categ_Oct_2021

- Faible chiffre d'affaire pour la catégorie 1 & 2 au mois d'Octobre 2021

### *Comparaison des chiffre d'affaire par mois sur chaque années*

In [ ]:
CAtempV2 = CAtemp
CAtempV2['year'] = pd.DatetimeIndex(CAtempV2['date']).year
CAtempV2['month'] = pd.DatetimeIndex(CAtempV2['date']).month
CAtempV2 = CAtempV2.groupby(by=['year','month']).sum()
CAtempV2 = CAtempV2.unstack('year').reset_index()
CAtempV2

- Il manque les données des 2 premier mois de l'année 2021
- Nous avons seulement les donnée des 2 premiers moi de l'année 2023

In [ ]:
CAtempV2.plot.area(x = 'month',stacked=False ,figsize=(15, 8))
plt.title("Etude du CA par mois sur les 3 années d'activité", size = 20)
plt.xlabel("Mois de l'année", size = 15)
plt.ylabel("Chiffre d'affaire", size = 15)
plt.show()

**2021**
- Données débutant à partir du mois de février
- Grosse baisse du CA au mois d'octobre
- Belle augmentation en général

**2022**
- CA qui augmente et diminue légèrement tout les 2 mois environ
- Stagnation du CA

**2023**
- Données des 2 premiers mois seulement
- Diminution du CA entre le 1er et le 2ème mois

In [ ]:
CA_moy_mois = CAtemp
CA_moy_mois['year'] = pd.DatetimeIndex(CA_moy_mois['date']).year
CA_moy_mois['month'] = pd.DatetimeIndex(CA_moy_mois['date']).month
CA_moy_mois = CA_moy_mois.groupby(by=['year','month']).sum().reset_index()
CA_moy_mois = round(CA_moy_mois.groupby(by=['year']).mean().reset_index())
CA_moy_mois

In [ ]:
CA_moy_mois.plot.bar(y='CA', x='year', figsize=(15, 8))
plt.title("Chiffre d'affaire moyen mensuel de chaque année", size = 20)
plt.ylabel("Chiffre d'affaire ", size = 15)
plt.xlabel("Année d'activité", size = 15)
#plt.savefig('test')
plt.show()

**Top 1**
- Année 2022 avec un chiffre d'affaire moyen par mois de plus de 500 000€

**Top 2**
- Année 2023 avec un chiffre d'affaire moyen par mois de proche de 500 000€

**Top 3**
- Année 2021 avec un chiffre d'affaire moyen légèrement infèrieur à celui de 2023

### *Chiffre d'affaire par année*

In [ ]:
CAyear = data.groupby(pd.Grouper(key='date', freq='Y')).sum().reset_index()
CAyear['date'] = pd.DatetimeIndex(CAyear['date']).year
CAyear = CAyear.rename(columns={'price':'CA'})
CAyear = CAyear[['date','CA']]
CAyear

In [ ]:
CAyear.plot.bar(y='CA', x='date', figsize=(15, 8))
plt.title("Chiffre d'affaire annuel", size = 20)
plt.ylabel("Chiffre d'affaire (en million)", size = 15)
plt.xlabel("Année d'activité", size = 15)
plt.show()

**2021**
- CA atteignant presque les 5 million d'euro

**2022**
- CA dépassant les 6 million d'euro

**2023**
- CA atteignant presque les 1 million d'euro (seulement les 2 premiers mois)

### *Chiffre d'affaire par catégorie*

In [ ]:
CAcateg = data.groupby(by=['categ']).sum()
CAcateg = CAcateg.rename(columns={'price':'CA'})
CAcateg = CAcateg[['CA']]
CAcateg

In [ ]:
colors = sns.color_palette("muted")
CAcateg.plot.pie(y = 'CA', colors = colors, figsize=(11, 11), autopct = "%1.1f%%")
plt.title("Répartition du chiffre d'affaire par catégorie", size = 20)
plt.legend(['Catégorie 0','Catégorie 1','Catégorie 2'], fontsize = 15)
plt.show()

**La catégorie qui effectue le plus gros chiffre d'affaire est la catégorie 1 !**

- La catégorie 0 est très proche de la catégorie 1 : une différence d'environ 2%
- La catégorie 2 est la catégorie qui fait le moin de chiffre d'affaire : néanmoin elle s'approche des 25%

### *Chiffre d'affaire par genre*

In [ ]:
CAgenre = data.groupby(by=['sex']).sum()
CAgenre = CAgenre.rename(columns={'price':'CA'})
CAgenre = CAgenre[['CA']]
CAgenre

In [ ]:
CAgenre.plot.pie(y = 'CA', colors = colors, figsize=(11, 11), autopct = "%1.1f%%")
plt.title("Répartition du chiffre d'affaire par genre", size = 20)
plt.legend(['Femme', 'Homme'], fontsize = 15)
plt.show()

**La majorité du chiffre d'affaire est réaliser par les hommes !**

Cependant nous somme trés proche d'une égalité du CA entre homme et femme

### Répartition du chiffre d'affaire

In [ ]:
CA_Client = data.groupby('client_id').sum()
CA_Client = CA_Client.rename(columns={'price':'CA'})
CA_Client = CA_Client['CA'].reset_index()
CA_Client.sort_values(by=["CA"], ascending=False)

**On remarque 4 clients avec un chiffre d'affaire générer trés important !**

- On concidère ces clients comme des clients professionnel

In [ ]:
# Courbe de Lorenz
def lorenz(variable, title):
    X = variable.values
    X = np.sort(X)
    
    # Indice de Gini
    def gini(array):
        array
        sorted_array = array.copy()
        sorted_array.sort()
        n = array.size
        coef_ = 2. / n
        const_ = (n + 1.) / n
        weighted_sum = sum([(i+1)*yi for i, yi in enumerate(sorted_array)])
        return coef_*weighted_sum/(sorted_array.sum()) - const_
    print('Incide de Gini :', round(gini(X), 2))
    
    # Courbe de Lorenz
    X_lorenz = X.cumsum() / X.sum()
    X_lorenz = np.insert(X_lorenz, 0, 0)
    # X_lorenz[0], X_lorenz[-1]
    y = np.arange(X_lorenz.size)/(X_lorenz.size-1)
    lorenz = pd.DataFrame()
    lorenz['X'] = pd.Series(X_lorenz)
    lorenz['Y'] = pd.Series(y)
    sns.scatterplot(data=lorenz, x='Y', y='X', marker='x')

    # Diagonale
    a = np.arange(0,1,.01)
    x = a
    y = a
    
    # Graphique
    sns.lineplot(x=x,y=y)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.title(title)
    plt.xlabel("Distribution des clients (%)")
    plt.ylabel("Cumul du chiffre d'affaire (%)")
    plt.show()
    
lorenz(CA_Client['CA'], 'Inégalités de CA entre tous les clients')

## <center>Analyse de tendance des produits

### *Chiffre d'affaire par produit*

In [ ]:
CAprod = data.groupby(by=['id_prod']).sum().reset_index()
CAprod = CAprod.rename(columns={'price':'CA'})
CAprod = CAprod[['id_prod','CA']]
print("Liste des produits et chiffre d'affaire effectuer :")
CAprod

### *Top 10 des tops produits*

In [ ]:
CAprod.sort_values(by=["CA"], ascending=False).head(10)

In [ ]:
top_10_CAprod = CAprod.sort_values(by=["CA"], ascending=False).head(10)
top_10_CAprod.sort_values(by=["CA"], ascending=True).plot.barh(x = 'id_prod',figsize=(15, 8))
plt.title("Top 10 des produits avec le plus gros CA", size = 20)
plt.ylabel("Produits", size = 15)
plt.xlabel("Chiffre d'affaire", size = 15)
plt.show()

Le produit 2_159 est un **Best-Seller** qui atteind presque les **100 000€** de chiffre d'affaire !

### *Top 10 des produits les plus vendu*

In [ ]:
Nbprod = data.groupby(by=['id_prod']).count().reset_index()
Nbprod = Nbprod.rename(columns={'price':'NbVendu'})
Nbprod = Nbprod[['id_prod','NbVendu']]
Nbprod.sort_values(by=['NbVendu'], ascending=False).head(10)

In [ ]:
top_10_vente = Nbprod.sort_values(by=['NbVendu'], ascending=False).head(10)
top_10_vente.sort_values(by=["NbVendu"], ascending=True).plot.barh(x = 'id_prod',figsize=(15, 8))
plt.title("Top 10 des produits les plus vendu", size = 20)
plt.ylabel("Produits", size = 15)
plt.xlabel("Nombre de vente", size = 15)
plt.show()

### *Top 10 des flop produits*

In [ ]:
CAprod.sort_values(by=["CA"], ascending=True).head(10)

In [ ]:
flop_10_CAprod = CAprod.sort_values(by=["CA"], ascending=True).head(10)
flop_10_CAprod.sort_values(by=["CA"], ascending=False).plot.barh(x = 'id_prod',figsize=(15, 8))
plt.title("Flop 10 des produits avec le moins de CA", size = 20)
plt.ylabel("Produits", size = 15)
plt.xlabel("Chiffre d'affaire", size = 15)
plt.show()

### *Produits les moins vendu*

In [ ]:
# Liste des produits avec moins de 10 exemplaires vendu

Nbprod[Nbprod['NbVendu']< 10]

**466 produits avec un nombre de vente infèrieur à 10 ventes !**

### *Moyenne de prix des categorie*

In [ ]:
MoyPrixCateg = data.groupby(by=['categ']).mean()
MoyPrixCateg = MoyPrixCateg[['price']].reset_index()
round(MoyPrixCateg)

In [ ]:
round(MoyPrixCateg).plot.bar(x='categ', y='price', figsize=(15, 8))
plt.title("Prix moyen d'un produit par catégorie", size = 20)
plt.ylabel("Prix (en €)", size = 15)
plt.xlabel("Catégorie", size = 15)
plt.show()

**Prix moyen des produits par catégories :**

- Categ 0 : 11€
- Categ 1 : 20€
- Categ 2 : 76€

### *Quantité de produits et  quantité de produits par catégorie*

In [ ]:
# Nombre total de produits différents vendu !

NbProdVendu = data['id_prod'].nunique()
print("Durant la période étudier", NbProdVendu,"produits différent ont été vendu !")

In [ ]:
# Répartition des produits par catégorie !

QteProdCateg = data.groupby('categ')['id_prod'].nunique()
QteProdCateg.plot.pie(colors = colors, figsize=(11, 11), autopct = "%1.1f%%")
plt.title("Répartition du nombre de produits vendu par catégorie", size = 20)
plt.legend(['Catégorie 0','Catégorie 1','Catégorie 2'], fontsize = 15)
plt.show()

- La majorité des produits vendu font parti de la catégorie 0, avec plus de 70% des ventes !

- La catégorie 1 est proche des 25%

- Un peu plus de 7% pour la catégorie 2

**En comparant la répartition des vente par catégorie ainsi que le prix moyen par catégorie, on peu constater que moins le prix moyen est élever plus les produits sont vendu !**

## <center>Analyse de tendance profil client

### *Moyenne d'age par catégorie*

In [ ]:
age_categ = data.groupby(by=['categ']).mean()
age_categ = age_categ.rename(columns={'age':'Moy age'})
age_categ = round(age_categ[['Moy age']])
age_categ.plot.bar(figsize=(15, 8))
plt.title("Moyenne d'age pour chaque catégories", size= 20)
plt.ylabel("Age", size= 15)
plt.xlabel("Catégorie", size= 15)
plt.show()
print(age_categ)

- La catégorie 0 a une moyenne d'age de : 45 ans
- La catégorie 1 a une moyenne d'age de : 49 ans
- La catégorie 2 a une moyenne d'age de : 27 ans

**Ici on remarque que la moyenne d'age de la catégorie 2 est la plus basse, donc on suppose que la catégorie 2 est une catégorie dédier à un public plus jeune !**

### *Chiffre d'affaire par age*

In [ ]:
ageCA = data.groupby(by='age').sum()
ageCA = ageCA.rename(columns={'price':'CA'})
ageCA = ageCA[['CA']].reset_index()

ageCA.plot.scatter(y= 'CA',x= 'age', figsize=(15, 8))
plt.title("Chiffre d'affaire en fonction de l'age des clients", size=20)
plt.xlabel("Age", size= 15)
plt.ylabel("Chiffre d'affaire", size= 15)
plt.show()

- Augmentation du CA en fonction de l'age : entre 20 et 45 ans
- Diminution du CA en fonction de l'age : à partir de 45 ans

### *Montant du panier par age*

In [ ]:
Age_Panier_Moy = data.groupby(['age','session_id']).sum()
Age_Panier_Moy = Age_Panier_Moy.groupby(['age']).mean()
Age_Panier_Moy = Age_Panier_Moy.rename(columns={'price':'Prix (en €)'})
Age_Panier_Moy = Age_Panier_Moy[['Prix (en €)']].reset_index()

Age_Panier_Moy.plot.scatter(y= 'Prix (en €)',x= 'age', figsize=(15, 8))
plt.title("Montant du panier moyen par age", size=20)
plt.xlabel("Age", size= 15)
plt.ylabel("Prix (en €)", size= 15)
plt.show()

**On distingue 3 groupes :**

- Les moins de 30 ans ont les plus gros panier moyen (environ 70€)

- Les 30, 50 ans ont environ un panier moyen de 33€

- Les 50 ans et plus ont environ un panier moyen de 25€

**Le montant le plus élever etant celui des personne ayant moin de 30 ans, nous supposons qu'ils achète majoritairement des produits de la catégorie 2, qui est la plus chère !**

**Et comme nous l'avons remarquer cette catégorie est dédier à un public jeune !**

### *Répartition des clients par groupe d'age*

**Réalisation des groupe d'age en fonction du montant du panier moyen par age**

In [ ]:
# Groupe des moins de 30 ans

group1 = data[data['Groupe'] == "- 30 ans"]
group1

In [ ]:
# Groupe des 30-50 ans

group2 = data[data['Groupe'] == "30-50 ans"]
group2

In [ ]:
# Groupe des 50-70 ans

group3 = data[data['Groupe'] == "50-70 ans"]
group3

In [ ]:
# Groupe des plus de 70 ans

group4 = data[data['Groupe'] == "+ 70 ans"]
group4

In [ ]:
datagroup = {'Groupe': ['Moins de 30 ans', '30-50 ans', '50-70 ans', 'Plus de 70 ans'],
             'Qte_Client': [group1['client_id'].nunique(), group2['client_id'].nunique(), group3['client_id'].nunique(), group4['client_id'].nunique()],
             'Femme': [group1[group1['sex']=='f']['client_id'].nunique(),group2[group2['sex']=='f']['client_id'].nunique(),group3[group3['sex']=='f']['client_id'].nunique(),group4[group4['sex']=='f']['client_id'].nunique()],
             'Homme': [group1[group1['sex']=='m']['client_id'].nunique(),group2[group2['sex']=='m']['client_id'].nunique(),group3[group3['sex']=='m']['client_id'].nunique(),group4[group4['sex']=='m']['client_id'].nunique()],
             'CA': [round(group1['price'].sum()),round(group2['price'].sum()),round(group3['price'].sum()),round(group4['price'].sum())]
            }

dfgroup = pd.DataFrame(datagroup)
dfgroup = dfgroup.set_index('Groupe')
dfgroup

In [ ]:
data[data['sex'] == "f"].nunique()

In [ ]:
dfgroup.plot.pie(y = 'Qte_Client', colors = colors, figsize=(11, 11), autopct = "%1.1f%%")
plt.title("Répartition des clients par tranche d'age", size=20)
plt.show()

- Le groupe des 30-50 ans est le groupe majoritaire avec presque 40%
- Avec presque 30% nous avons le groupe des 50-70 ans


In [ ]:
dfgroup[['Homme', 'Femme']].plot.bar( figsize=(15, 8))
plt.title("Comparaison du nombre de clients par genre en fonction de la tranche d'age", size = 20)
plt.ylabel("Nombre de clients")
plt.xlabel("Tranches d'age")
plt.show()

- Dans chaque groupe d'age nous avons plus de femme que d'homme

- Nous avons environ de centaine de femme de plus que d'homme dans chaque groupe

In [ ]:
dfgroup.plot.pie(y = 'CA', colors = colors, figsize=(11, 11), autopct = "%1.1f%%")
plt.title("Répartition du chiffre d'affaire par tranche d'age", size=20)
plt.show()

# <center>**Demandes de Julie**
---

In [ ]:
# Suppression des clients professionnel

client_pro_1 = data[data['client_id'] == 'c_1609']
client_pro_2 = data[data['client_id'] == 'c_4958']
client_pro_3 = data[data['client_id'] == 'c_6714']
client_pro_4 = data[data['client_id'] == 'c_3454']

data = data.drop(client_pro_1.index)
data = data.drop(client_pro_2.index)
data = data.drop(client_pro_3.index)
data = data.drop(client_pro_4.index)

In [ ]:
client_pro_4[client_pro_4['Mois-Année'] == '8-2021']

In [ ]:
# Part du chiffre d'affaire et chiffre d'affaire des clients professionnel

CA_Pro = client_pro_1['price'].sum() + client_pro_2['price'].sum() + client_pro_3['price'].sum() + client_pro_4['price'].sum()
Part_CA = round((CA_Pro / CA)*100,2)
print("Les 4 clients professionnels on générer",Part_CA,"% du chiffre d'affaire, soit",CA_Pro,"€")

**On supprime les clients professionnels pour éviter des résultats faussé !**

In [ ]:
# Fonction de vérification de la p-value

def test_p_values(p):
    if p > 0.05:
        print("Variables indépendantes, car p-value > 5% ")
        print("Donc les 2 variables ne sont pas corréler !")
        print("p-value :",p)
    else:
        print("On rejète H0, car p-value < 5%")
        print("Donc les 2 variables sont corréler !")
        print("p-value :",p)

### *Corrélation du genre et de la catégorie*

In [ ]:
vente_sex_categ = data.groupby(by=['sex','categ']).count()
vente_sex_categ = vente_sex_categ.rename(columns={'price':'nb_prod'})
vente_sex_categ = vente_sex_categ[['nb_prod']]
vente_sex_categ = vente_sex_categ.unstack('sex')
vente_sex_categ = vente_sex_categ.rename(index={ 0 :'Categ 0', 1 :'Categ 1', 2 :'Categ 2'})
vente_sex_categ

In [ ]:
vente_sex_categ.plot.pie(subplots=True, colors = colors, figsize=(11, 11), autopct = "%1.1f%%")

plt.subplot(1,2,1)
plt.title('Achats des femmes')

plt.subplot(1,2,2)
plt.title('Achats des hommes')

plt.show()

In [ ]:
# Le lien entre le genre d'un client et les catégories de livres achetés - Analyse QUALITATIF / QUALITATIF
# Tableau de contingence

X = "categ"
Y = "sex"

sex_categ = data[[X,Y]].pivot_table(index=X,columns=Y,aggfunc=len,margins=True,margins_name="Total")
sex_categ

In [ ]:
# HeatMap

tx = sex_categ.loc[:,["Total"]]
ty = sex_categ.loc[["Total"],:]
n = len(data)
indep = tx.dot(ty) / n

c = sex_categ.fillna(0) # On remplace les valeurs nulles par 0
measure = (c-indep)**2/indep
xi_n = measure.sum().sum()
table = measure/xi_n
sns.heatmap(table.iloc[:-1,:-1],annot=c.iloc[:-1,:-1])
plt.title("Tableau de contingence genre/catégorie coloré")
plt.show()

In [ ]:
# Le lien entre le genre d'un client et les catégories de livres achetés - Analyse QUALITATIF / QUALITATIF

Khi2_obs, p_value, ddl, effectif_theorique = chi2_contingency(sex_categ)
print(Khi2_obs)
test_p_values(p_value)

**Synthèse de la corrélation entre le genre et la catégorie**

Dans un premier temps à la vue du premier graphique nous ne voyeont aucune différence entre le genre masculin et le genre féminin, ce qui nous laisse penser qu'il n'y a aucune corrélation.
<br/>Les répartitions sont casiment identique !
<br/>Mais en regardant le deuxième graphique on s'aperçoit d'une légère différence.<br/>
<br/>Nous sommes face à 2 variables quantitatives !
<br/>Cependant aprés avoir effectuer le test statistique Khi2, nous obtenons une p-value infèrieur à 0,05.
<br/>Ce qui signifie que l'on rejète l'hypothèse nulle, selon laquelle les 2 variables sont indépendante, donc il **existe** bien une corrélation entre ces 2 variables !

### *Corrélation de l'age et du montant total des achats*

In [ ]:
# Montant total des achats par clients

Age_Panier = data.groupby(['client_id', 'age']).sum()
#Age_Panier = data.groupby(['age']).sum()
Age_Panier = Age_Panier.rename(columns={'price':'Montant Total'})
Age_Panier = Age_Panier.reset_index()
Age_Panier = Age_Panier[['age', 'Montant Total']]

slope, intercept, r_value, p_value, std_err = stats.linregress(Age_Panier['age'], Age_Panier['Montant Total'])
def predict(x):
   return slope * x + intercept

Age_Panier.plot.scatter(y= 'Montant Total',x= 'age', figsize=(15, 8))
plt.title("Montant total des achats par clients en fonction de leurs âge", size=20)
plt.ylabel("Montant (en €)", size=15)
plt.xlabel("Age", size=15)
fitLine = predict(Age_Panier['age'])
plt.plot(Age_Panier['age'], fitLine, c='r')
plt.show()


In [ ]:
Age_Panier = Age_Panier.groupby(['age']).sum()
Age_Panier = Age_Panier.reset_index()
Age_Panier.plot.scatter(y= 'Montant Total',x= 'age', figsize=(15, 8))

slope, intercept, r_value, p_value, std_err = stats.linregress(Age_Panier['age'], Age_Panier['Montant Total'])
def predict(x):
   return slope * x + intercept

plt.title("Montant total des achats par âge", size=20)
plt.ylabel("Montant (en €)", size=15)
plt.xlabel("Age", size=15)
fitLine = predict(Age_Panier['age'])
plt.plot(Age_Panier['age'], fitLine, c='r')
plt.show()

- Le montant des achats total des personnes agé de 19 ans dépasse les **600 000 €**
- Tendance à l'augmentation entre 20 et 45 ans
- Tendance à la baisse à partir de 45 ans

- L'analyse de régression linéaire a montré une association négative entre les deux variables

In [ ]:
# Le lien entre l'âge des clients et le montant total des achats - Analyse QUANTITATIF / QUANTITATIF
# Calcul du coefficient de Pearson

corr, p = pearsonr(Age_Panier['age'], Age_Panier['Montant Total'])
print('Pearsons correlation: %.3f' % corr)
print("Nous voyons ici qu'il y'a une forte corrélation, car le coefficient est proche de -1 !")
test_p_values(p)

**Synthèse de la corrélation entre l'age et le montant total des achats**

Pour commencer, nous remarquons des valeurs extreme que nous écartons par la suite afin d'avoir un résultat le plus réaliste possible.
<br/>En effet le test utiliser est un test paramétrique, donc sensible aux outliers.
<br/>A première vue des graphique nous apercevons une corrélation !<br/>
<br/>Nous sommes face à 2 variables quantitatives !
<br/>Afin de vérifier cette corrélation, nous avons éffectuer un test statistique de Pearson, qui nous donne un coeficient de corrélation ainsi qu'une p-value.
<br/>Nous obtenons un coéfficient de corrélation proche de -1, donc il y a corrélation entre ces 2 variables !
<br/>La p-value obtenue est infèrieur à 0,05 ce qui confirme cette corrélation.

### *Corrélation de la tranche d'age et du montant total des achats*

In [ ]:
Montant_Total_Client_Groupe = data.groupby(['client_id', 'Groupe']).sum()
Montant_Total_Client_Groupe = Montant_Total_Client_Groupe.rename(columns={'price':'Montant Total'})
Montant_Total_Client_Groupe = Montant_Total_Client_Groupe.reset_index()


# Ordonner les groupes par ordre croissant
order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
Montant_Total_Client_Groupe = Montant_Total_Client_Groupe.reindex(Montant_Total_Client_Groupe['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)

X = "Groupe" # qualitative
Y = "Montant Total" # quantitative
modalites = Montant_Total_Client_Groupe[X].unique()
groupes = []
for m in modalites:
    groupes.append(Montant_Total_Client_Groupe[Montant_Total_Client_Groupe[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}


plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution du montant total des achat par groupe d'age")
plt.ylabel("Montant (en €)")
plt.xlabel("Groupe d'Age")
plt.show()

In [ ]:
Montant_Total_Client_Groupe

In [ ]:
import scipy.stats as stats

A = Montant_Total_Client_Groupe['Montant Total'][Montant_Total_Client_Groupe['Groupe'] == '- 30 ans']
B = Montant_Total_Client_Groupe['Montant Total'][Montant_Total_Client_Groupe['Groupe'] == '30-50 ans']
C = Montant_Total_Client_Groupe['Montant Total'][Montant_Total_Client_Groupe['Groupe'] == '50-70 ans']
D = Montant_Total_Client_Groupe['Montant Total'][Montant_Total_Client_Groupe['Groupe'] == '+ 70 ans']

#perform Bartlett's test 
stats.bartlett(A, B, C,D)

In [ ]:
# Le lien entre la tranche d'âge des clients et le montant total des achats - Analyse QUALITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='Montant Total', between='Groupe', data=Montant_Total_Client_Groupe)

**Synthèse de la corrélation entre la tranche d'age et le montant total des achats**

On observe une moyenne ainsi qu'une variation asser différente entre les différents groupe d'age, ce qui nous insite à penser qu'il existe une corrélation non-linéaire entre les 2 variables !<br/>
<br/>Nous somme face à une variable quantitative et une variable qualitative !
<br/>Nous avons effectuer le teste statistique Welch's ANOVA, qui nous permet de calculer le rapport de corrélation ainsi que la p-value.
<br/>On obtient un rapport de corrélation proche de 0, mais une p-value infèrieur à 0,05.
<br/>Il existe donc bien une corrélation entre ces 2 variables !

### *Corrélation entre l'age des clients et la fréquence d'achat*

In [ ]:
# Méthode n°1
# Calcul du nombre d'achat moyen mensuelle par client pour chaque ages

Freq_Age = data
Freq_Age['year'] = pd.DatetimeIndex(Freq_Age['date']).year.astype(str)
Freq_Age['month'] = pd.DatetimeIndex(Freq_Age['date']).month.astype(str)
Freq_Age['Mois-Année'] = Freq_Age['month']+'-'+Freq_Age['year']
Freq_Age = Freq_Age[['client_id','session_id','id_prod','age', 'Mois-Année']]
Freq_Age = Freq_Age.groupby(['client_id','session_id','Mois-Année','age']).count()
Freq_Age = Freq_Age.groupby(['client_id','age','Mois-Année']).count().reset_index()
Freq_Age = round(Freq_Age.groupby(['client_id','age']).sum()/24).reset_index()

#Freq_Age = round(Freq_Age.groupby('age').mean().reset_index(),2)

slope, intercept, r_value, p_value, std_err = stats.linregress(Freq_Age['age'], Freq_Age['id_prod'])
def predict(x):
   return slope * x + intercept

Freq_Age.plot.scatter(x='age', y='id_prod',figsize=(15, 8))
plt.title("Nombre de produits mensuel acheté par client en fonction de l'âge", size=20)
plt.xlabel("Age", size = 15)
plt.ylabel("Nombre de produits", size = 15)
fitLine = predict(Freq_Age['age'])
plt.plot(Freq_Age['age'], fitLine, c='r')
plt.show()

In [ ]:
# Le lien entre l'âge des clients et la fréquence des achats - Analyse QUALITATIF / QUANTITATIF
# Test de Pearson

# H0 = Les 2 variables ne sont pas corréler !
# H1 = Les 2 variable sont corréler 

corr, p = pearsonr(Freq_Age['age'], Freq_Age['id_prod'])
print('Pearsons correlation: %.3f' % corr)
test_p_values(p)

In [ ]:
# Méthode n°2
# Calcul du nombre de produit acheter par client sur la pèriode étudier en fonction de l'age

Freq_Achat_Age = data.groupby(['client_id','age']).nunique()
Freq_Achat_Age = Freq_Achat_Age[['session_id']].reset_index()
Freq_Achat_Age = Freq_Achat_Age.rename(columns={'session_id':'Nb_Achat'})

slope, intercept, r_value, p_value, std_err = stats.linregress(Freq_Achat_Age['age'], Freq_Achat_Age['Nb_Achat'])
def predict(x):
   return slope * x + intercept

Freq_Achat_Age.plot.scatter(x='age', y='Nb_Achat',figsize=(15, 8))
plt.title("Nombre d'achat total par client en fonction de l'age ", size=20)
plt.xlabel("Age", size = 15)
plt.ylabel("Nombre d'achats", size = 15)
fitLine = predict(Freq_Achat_Age['age'])
plt.plot(Freq_Achat_Age['age'], fitLine, c='r')
plt.show()

In [ ]:
Freq_Achat_Moy_Age = round(Freq_Achat_Age.groupby('age').mean().reset_index())

slope, intercept, r_value, p_value, std_err = stats.linregress(Freq_Achat_Moy_Age['age'], Freq_Achat_Moy_Age['Nb_Achat'])
def predict(x):
   return slope * x + intercept

Freq_Achat_Moy_Age.plot.scatter(x='age', y='Nb_Achat',figsize=(15, 8))
plt.title("Nombre d'achat moyen des clients par age", size=20)
plt.xlabel("Age")
plt.ylabel("Nombre d'achats") 
fitLine = predict(Freq_Achat_Moy_Age['age'])
plt.plot(Freq_Achat_Moy_Age['age'], fitLine, c='r')
plt.show()

- L'analyse de régression linéaire montre une association positive entre les deux variables

In [ ]:
# Le lien entre l'âge des clients et la fréquence des achats - Analyse QUALITATIF / QUANTITATIF
# Test de Pearson

# H0 = Les 2 variables ne sont pas corréler !
# H1 = Les 2 variable sont corréler 

corr, p = pearsonr(Freq_Achat_Age['age'], Freq_Achat_Age['Nb_Achat'])
print('Pearsons correlation: %.3f' % corr)
test_p_values(p)

**Synthède de la corrélation entre l'age et la fréquence d'achat**

J'ai décider d'étudier cette corrélation en utilisant 2 méthode différentes !<br/>
<br/><br/>
*Méthode n°1 : Nombre de produits moyen acheté par mois*
<br/><br/>
Au vue du graphique, on appercois des différences asser légère en par rapport à l'age, ce qui laisse penser à une corrélation !
<br/><br/>
Nous sommes face à 2 variables quantitative !
<br/>
Pour vérifier cette théorie, on utilise le teste de Pearson.
<br/>
Grace à ce teste nous comprenons qu'il y a une corrélation entre les 2 variables, car le coéfficient de corrélation obtenue est proche de 0.160 mais la p-value est infèrieur à 0.05 !
<br/><br/>
*Méthode n°2 : Nombre d'achat total*
<br/><br/>
Pour commencer on s'appercoit qu'il y a 4 valeurs extreme que l'on écarte de l'étude, afin d'avoir par la suite un résultat clair au teste statistique.
<br/>
On s'appersoit grace aux graphique que nous avons une légère tendence du nombre d'achat en fonction de l'age. Donc il y a probablement une corrélation.
<br/><br/>
Nous sommes face à 2 variables quantitatives !
<br/>
Avec le teste de Pearson, nous allons pouvoir savoir si il y a en effet une corrélation.
<br/>
Les resultats nous apprènent qu'il y a effectivement une légère corrélation entre les 2 variables, nous constatons cele grace à la p-value qui est infèrieur à 0.05, malgré le coéfficient de corrélation proche de 0 !

### *Corrélation entre la tranche d'age des clients et la fréquence d'achat*

In [ ]:
# Méthode n°1
# Etude de la frequence d'achat grace au nombre moyen de produits acheter par mois sur la durée total étudier

Freq_Groupe_Age = data
Freq_Groupe_Age['year'] = pd.DatetimeIndex(Freq_Groupe_Age['date']).year.astype(str)
Freq_Groupe_Age['month'] = pd.DatetimeIndex(Freq_Groupe_Age['date']).month.astype(str)
Freq_Groupe_Age['Mois-Année'] = Freq_Groupe_Age['month']+'-'+Freq_Groupe_Age['year']
Freq_Groupe_Age = Freq_Groupe_Age[['client_id','session_id','id_prod','Groupe', 'Mois-Année']]
Freq_Groupe_Age= Freq_Groupe_Age.groupby(['client_id','session_id','Mois-Année','Groupe']).count()
Freq_Groupe_Age = Freq_Groupe_Age.groupby(['client_id','Groupe','Mois-Année']).count().reset_index()
Freq_Groupe_Age = round(Freq_Groupe_Age.groupby(['client_id','Groupe']).sum()/24).reset_index()

# Ordonner les groupes par ordre croissant
order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
Freq_Groupe_Age = Freq_Groupe_Age.reindex(Freq_Groupe_Age['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)

X = "Groupe" # qualitative
Y = "id_prod" # quantitative
modalites = Freq_Groupe_Age[X].unique()
groupes = []
for m in modalites:
    groupes.append(Freq_Groupe_Age[Freq_Groupe_Age[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}
    
plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution du nombre de produits mensuels acheter par groupe d'âge")
plt.ylabel("Nombre de produits")
plt.xlabel("Groupe d'Age")
plt.show()

In [ ]:
# Le lien entre la tranche d'âge des clients et la fréquence des achats - Analyse QUALITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='id_prod', between='Groupe', data=Freq_Groupe_Age)

In [ ]:
# Méthode n°2
# Etude de la frequence d'achat grace au nombre d'achat sur la durée total étudier

Freq_Achat_Groupe_Age = data.groupby(['client_id','Groupe']).nunique()
Freq_Achat_Groupe_Age = Freq_Achat_Groupe_Age[['session_id']].reset_index()
Freq_Achat_Groupe_Age = Freq_Achat_Groupe_Age.rename(columns={'session_id':'Nb_Achat'})
# On enlève les outliers
Freq_Achat_Groupe_Age = Freq_Achat_Groupe_Age[Freq_Achat_Groupe_Age['Nb_Achat'] < 2000]

# Ordonner les groupes par ordre croissant
order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
Freq_Achat_Groupe_Age = Freq_Achat_Groupe_Age.reindex(Freq_Achat_Groupe_Age['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)

X = "Groupe" # qualitative
Y = "Nb_Achat" # quantitative
modalites = Freq_Achat_Groupe_Age[X].unique()
groupes = []
for m in modalites:
    groupes.append(Freq_Achat_Groupe_Age[Freq_Achat_Groupe_Age[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}
    
plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution du nombre d'achat total par groupe d'age")
plt.ylabel("Nombre d'achats")
plt.xlabel("Groupe d'Age")
plt.show()

In [ ]:
# Le lien entre la tranche d'âge des clients et la fréquence des achats - Analyse QUALITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='Nb_Achat', between='Groupe', data=Freq_Achat_Groupe_Age)

**Synthède de la corrélation entre le groupe d'age et la fréquence d'achat**

On réalise les 2 même methodes que précedement !
<br/><br/>
*Méthode n°1 : Nombre de produits moyen acheté par mois*
<br/><br/>
On vois différentes variation et différentes moyenne en fonction du groupe d'age, ce qui nous laisse penser à une corrélation entre les 2 variables !
<br/><br/>
Nous sommes face à 1 variable quantitative et 1 variable qualitative !
<br/>
Le test Welch's ANOVA nous donne un rapport de corrélation proche de 0 mais une p-value infèrieur à 0,05.
<br/>
Donc nous avons bien une corrélation entre ces 2 variables !
<br/><br/>
*Méthode n°2 : Nombre d'achat total*
<br/><br/>
On vois différentes variation et différentes moyenne en fonction du groupe d'age, ce qui nous laisse penser à une corrélation entre les 2 variables !
<br/><br/>
Nous sommes face à 1 variable quantitative et 1 variable qualitative !
<br/>
Le test Welch's ANOVA nous donne un rapport de corrélation proche de 0 mais une p-value infèrieur à 0,05.
<br/>
Donc nous avons bien une corrélation entre ces 2 variables !

### *Corrélation entre l'age des clients et la taille du panier moyen*

In [ ]:
Age_Taille_Panier_Moy = data.groupby(['age', 'session_id']).count()
Age_Taille_Panier_Moy = round(Age_Taille_Panier_Moy.groupby('age').mean())
Age_Taille_Panier_Moy = Age_Taille_Panier_Moy.rename(columns = {'id_prod':'nb_prod'})
Age_Taille_Panier_Moy = Age_Taille_Panier_Moy[['nb_prod']]
Age_Taille_Panier_Moy = Age_Taille_Panier_Moy.reset_index()

slope, intercept, r_value, p_value, std_err = stats.linregress(Age_Taille_Panier_Moy['age'], Age_Taille_Panier_Moy['nb_prod'])
def predict(x):
   return slope * x + intercept

Age_Taille_Panier_Moy.plot.scatter(x= 'age', y= 'nb_prod', figsize=(15, 8))
plt.title("Taille du panier moyen des clients par age", size=20)
plt.ylabel("Nombre de produits", size=15)
plt.xlabel("Age", size=15)
fitLine = predict(Age_Taille_Panier_Moy['age'])
plt.plot(Age_Taille_Panier_Moy['age'], fitLine, c='r')
plt.show()

- L'analyse de régression linéaire montre une association négative entre les deux variables

In [ ]:
# Le lien entre l'âge des clients et la taille du panier moyen - Analyse QUANTITATIF / QUANTITATIF
# Calcul du coefficient de Pearson

corr, p = pearsonr(Age_Taille_Panier_Moy['age'], Age_Taille_Panier_Moy['nb_prod'])
print('Pearsons correlation: %.3f' % corr)
print("Nous voyons ici qu'il y'a une importante corrélation !")
print("p-value : ",p)

**Synthède de la corrélation entre l'age et la taille moyen du panier**

Sur le graphique on peut voir une différence entre les ages, ce qui nous laisse penser à une corrélation.
<br/><br/>
Nous sommes face à 2 variables quantitative !
<br/>
Grace aux resultats du test de Pearson nous obtenons un coéfficient de corrélation proche de -1, donc nous avons une corrélation.
<br/>
De plus nous avons une p-value infèrieur à 0.05 qui vient confirmé cette corrélation !

### *Corrélation entre l'age des clients et le montant du panier moyen*

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(Age_Panier_Moy['age'], Age_Panier_Moy['Prix (en €)'])
def predict(x):
   return slope * x + intercept


Age_Panier_Moy.plot.scatter(y= 'Prix (en €)',x= 'age', figsize=(15, 8))
plt.title("Montant du panier moyen des clients par age", size=20)
plt.ylabel("Montant (en €)", size=15)
plt.xlabel("Age", size=15)
fitLine = predict(Age_Panier_Moy['age'])
plt.plot(Age_Panier_Moy['age'], fitLine, c='r')
plt.show()

- L'analyse de régression linéaire montre une association négative entre les deux variables

In [ ]:
# Le lien entre l'âge des clients et le montant du panier moyen - Analyse QUANTITATIF / QUANTITATIF
# Calcul du coefficient de Pearson

corr, p = pearsonr(Age_Panier_Moy['age'], Age_Panier_Moy['Prix (en €)'])
print('Pearsons correlation: %.3f' % corr)
print("Nous voyons ici qu'il y'a une importante corrélation !")
print("p-value : ",p)

**Synthède de la corrélation entre l'age et le montant moyen du panier**

Sur le graphique on peut voir une différence entre les ages, ce qui nous laisse penser à une corrélation.
<br/><br/>
Nous sommes face à 2 variables quantitative !
<br/>
Grace aux resultats du test de Pearson nous obtenons un coéfficient de corrélation proche de -1, donc nous avons une corrélation.
<br/>
De plus nous avons une p-value infèrieur à 0.05 qui vient confirmé cette corrélation !

### *Corrélation entre la tranche d'age des clients et la taille moyen du panier*

In [ ]:
nbprod = data.groupby(['Groupe', 'session_id']).count()
nbprod = nbprod[['id_prod']].reset_index()


# Ordonner les groupes par ordre croissant
order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
nbprod = nbprod.reindex(nbprod['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)

X = "Groupe" # qualitative
Y = "id_prod"# quantitative

modalites = nbprod[X].unique()
groupes = []
for m in modalites:
    groupes.append(nbprod[nbprod[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}
    
plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution de la taille du panier moyen par groupe d'age")
plt.ylabel("Nombre de produits")
plt.xlabel("Groupe d'Age")
plt.show()

In [ ]:
# Le lien entre la tranche d'âge des clients et la taille du panier moyen - Analyse QUANTITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='id_prod', between='Groupe', data=nbprod)

**Synthède de la corrélation entre le groupe d'age et la taille moyen du panier**

Sur le graphique on s'appercois qu'il y a 3 groupe dont la variation est trés similaire, cependant toutes les moyennes sont différentes.
<br/>
On pourrais donc s'attendre à une légère corrélation !
<br/><br/>
Nous sommes face à 1 variable quantitative et 1 variable qualitative !
<br/>
Le test Welch's ANOVA nous confirme cette légère corrélation grace à un p-value infèrieur à 0.05, malgré le rapport de corrélation proche de 0.

### *Corrélation entre la tranche d'age des clients et le montant moyen du panier*

In [ ]:
Panier_Moyen_Client_Groupe = data.groupby(['Groupe','session_id']).sum().reset_index()
Panier_Moyen_Client_Groupe = Panier_Moyen_Client_Groupe[['Groupe','session_id','price']]

# Ordonner les groupes par ordre croissant
order = ['- 30 ans', '30-50 ans', '50-70 ans', '+ 70 ans']
Panier_Moyen_Client_Groupe = Panier_Moyen_Client_Groupe.reindex(Panier_Moyen_Client_Groupe['Groupe'].map(dict(zip(order, range(len(order))))).sort_values().index)

X = "Groupe" # qualitative
Y = "price" # quantitative
modalites = Panier_Moyen_Client_Groupe[X].unique()
groupes = []
for m in modalites:
    groupes.append(Panier_Moyen_Client_Groupe[Panier_Moyen_Client_Groupe[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}
    
plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution du montant moyen du panier par groupe d'age")
plt.ylabel("Montant (en €)")
plt.xlabel("Groupe d'Age")
plt.show()

In [ ]:
# Le lien entre la tranche d'âge des clients et le montant du panier moyen - Analyse QUALITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='price', between='Groupe', data=Panier_Moyen_Client_Groupe)

**Synthède de la corrélation entre le groupe d'age et le montant moyen du panier**

On appercoit directement des différences significative sur les 3 groupes, avec une moyenne et une variation différente.
<br/>
Donc nous pouvons penser qu'il y'a une forte corrélation entre les 2 variables !
<br/><br/>
Nous sommes face à 1 variable quantitative et 1 variable qualitative !
<br/>
Le test Welch's ANOVA nous confirme cette corrélation grace à un p-value infèrieur à 0.05, malgré le rapport de corrélation proche de 0.
<br/>
Alors nous n'avons pas une forte corrélation !

### *Corrélation entre l'age des clients et les categories de livres achetés*

In [ ]:
X = "categ" # qualitative
Y = "age" # quantitative
modalites = data[X].unique()
groupes = []
for m in modalites:
    groupes.append(data[data[X]==m][Y])

# Propriétés graphiques (pas très importantes)    
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}
    
plt.boxplot(groupes, labels=modalites, showfliers=False, medianprops=medianprops, 
            vert=True, patch_artist=True, showmeans=True, meanprops=meanprops)
plt.title("Distribution des ages par catégorie")
plt.ylabel("Age")
plt.xlabel("Catégories")
plt.show()

In [ ]:
# Le lien entre la tranche d'âge des clients et le montant du panier moyen - Analyse QUALITATIF / QUANTITATIF
# Test Welch's ANOVA (paramétrique)

pg.welch_anova(dv='age', between='categ', data=data)

**Synthède de la corrélation entre l'age et les catégories de livres achetés**

En observant le premier graphique on s'appercoit que chaque catégorie a sa tendance en fonction de l'age, et donc on en conclue qu'il y aurait potentielement une corrélation entre les 2 variables.
<br/><br/>
Nous somme face à 2 variable quantitative !
<br/>
Aprés avoir effectuer le teste de Pearson pour le 3 catégorie de livres, on obtient les mêmes résultats.
<br/>
Les coéfficients de corrélation sont asser proche de -1 et on a une p-value infèrieur à 0,05.
<br/>
Ce qui nous confirme la corrélation entre les 2 variables !

### *Corrélation entre le groupe d'age des clients et les categories de livres achetés*

In [ ]:
# Le lien entre le groupe d'age des clients et les categories de livres achetés - Analyse QUALITATIF / QUALITATIF

X = "categ"
Y = "Groupe"

groupe_categ = data[[X,Y]].pivot_table(index=X,columns=Y,aggfunc=len,margins=True,margins_name="Total")
groupe_categ

groupe_categ

In [ ]:
tx = groupe_categ.loc[:,["Total"]]
ty = groupe_categ.loc[["Total"],:]
n = len(data)
indep = tx.dot(ty) / n

c = groupe_categ.fillna(0) # On remplace les valeurs nulles par 0
measure = (c-indep)**2/indep
xi_n = measure.sum().sum()
table = measure/xi_n
sns.heatmap(table.iloc[:-1,:-1],annot=c.iloc[:-1,:-1])
plt.show()

In [ ]:
# Le lien entre  le groupe d'age des clients et les categories de livres achetés - Analyse QUALITATIF / QUALITATIF

Khi2_obs, p_value, ddl, effectif_theorique = chi2_contingency(groupe_categ)
print(Khi2_obs)
test_p_values(p_value)

**Synthède de la corrélation entre l'age et les catégories de livres achetés**

Dans un premier temps en regardant le tableau de contingence on s'appercois qu'il y a des différence en fonction du groupe d'age.
<br/>
Ensuite en regardant le graphique on confirme cette différence, qui nous laisse penser à une corrélation.
<br/><br/>
Nous sommes face à 2 variable qualitative !
<br/>
Le teste de Chi-2 nous confirme une corrélation des variables, grace à une p-value infèrieur à 0,05 !